In [1]:
import numpy as np 
import pandas as pd 
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as functional
from torchvision import datasets
from torch.utils.data import DataLoader

In [2]:
batch_size = 64

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5,9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [3]:
train_dataset = datasets.ImageFolder(
    root='/kaggle/input/rating-opencv-emotion-images/Images/train',
    transform=train_transform
)

validation_dataset = datasets.ImageFolder(
    root='/kaggle/input/rating-opencv-emotion-images/Images/validation',
    transform=valid_transform
)

In [4]:
train_loader = DataLoader(
 train_dataset, batch_size=batch_size, shuffle=True, 
 num_workers=4, pin_memory=True
)

valid_loader = DataLoader(
    validation_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True
)

In [5]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 28709
    Root location: /kaggle/input/rating-opencv-emotion-images/Images/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
               RandomHorizontalFlip(p=0.5)
               RandomVerticalFlip(p=0.5)
               GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5))
               RandomRotation(degrees=[30.0, 70.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [6]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(256, 7)
        self.pool = nn.MaxPool2d(2, 2)
        
    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = self.pool(functional.relu(self.conv3(x)))
        x = self.pool(functional.relu(self.conv4(x)))
        
        bs, _, _, _ = x.shape
        x = functional.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
        x = self.fc1(x)
        return x
    


In [7]:
lr = 1e-3
epochs = 30

device = ('cuda' if torch.cuda.is_available() else 'cpu')
image_classifier = ImageModel().to(device)
optimizer = optim.Adam(image_classifier.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

device, image_classifier

('cuda',
 ImageModel(
   (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1))
   (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
   (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
   (conv4): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
   (fc1): Linear(in_features=256, out_features=7, bias=True)
   (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 ))

In [8]:
total_params = sum(p.numel() for p in image_classifier.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in image_classifier.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

948,807 total parameters.
948,807 training parameters.


In [9]:
def train(model, trainloader, optimizer, criterion):
    model.train()
    print('Training torch classifier')
    train_running_loss = 0.0
    train_running_correct = 0